In [1]:
from pyspark.sql import SparkSession
from delta import DeltaTable
from dotenv import load_dotenv
import os

In [14]:
load_dotenv('example.env')

spark = SparkSession.builder \
    .appName("MinIO Delta Lake Ingestion") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.1.0") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.endpoint", f"{os.environ['S3_ENDPOINT_URL']}") \
    .config("spark.hadoop.fs.s3a.access.key", f"{os.environ['S3_ACCESS_KEY']}") \
    .config("spark.hadoop.fs.s3a.secret.key", f"{os.environ['S3_SECRET_KEY']}") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.delta.merge.repartitionBeforeWrite.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [15]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Define schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True)
])

# Create dummy data
data = [(1, "Alice"), (2, "Bob"), (3, "Charlie")]

# Create DataFrame
df = spark.createDataFrame(data, schema)
df.show()

+---+-------+
| id|   name|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3|Charlie|
+---+-------+



In [17]:
df.write.format("delta").mode("overwrite").save("s3a://datalake/bronze/dummy")


Py4JJavaError: An error occurred while calling o130.save.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:582)
	at org.apache.spark.sql.delta.DeltaLog$.forTable(DeltaLog.scala:492)
	at org.apache.spark.sql.delta.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:151)
	at org.apache.spark.sql.execu